# ETL 

In [10]:
#Libraries
import os
import sqlite3
import pandas as pd
import numpy as np 
from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt

# Data Load

In [11]:
# Configuration and Setup 

# Define the path to the directory containing the Olist dataset
data_path = r'C:\Users\win\Desktop\PROJETO\dados'

# Get a list of all CSV file names in the directory
csv_files = [f for f in os.listdir(data_path) if f.endswith('.csv')]


# Load CSVs into an In-Memory SQLite Database 

# Create a connection to an in-memory SQLite database.
conn = sqlite3.connect(':memory:')
for csv_file in csv_files:
    # Use the file name (without the .csv extension) as the table name
    table_name = os.path.splitext(csv_file)[0]
    file_path = os.path.join(data_path, csv_file)
    
    # Read the CSV file into a temporary DataFrame
    df_temp = pd.read_csv(file_path)
    # Write the DataFrame to a SQL table
    df_temp.to_sql(table_name, conn, index=False, if_exists='replace')
    
    print(f'- Table "{table_name}" created with {df_temp.shape[0]} rows.')


- Table "olist_customers_dataset" created with 99441 rows.
- Table "olist_geolocation_dataset" created with 1000163 rows.
- Table "olist_orders_dataset" created with 99441 rows.
- Table "olist_order_items_dataset" created with 112650 rows.
- Table "olist_order_payments_dataset" created with 103886 rows.
- Table "olist_order_reviews_dataset" created with 100000 rows.
- Table "olist_products_dataset" created with 32951 rows.
- Table "olist_sellers_dataset" created with 3095 rows.
- Table "product_category_name_translation" created with 71 rows.


# SQL 

In [12]:
# This SQL query joins all the individual tables into a single, unified dataset.
join_query = '''
SELECT 
    c.*,
    o.*,
    p.*,
    r.*,
    i.*,
    x.*
    
FROM
    olist_customers_dataset c
JOIN 
    olist_orders_dataset o ON c.customer_id = o.customer_id
JOIN 
    olist_order_payments_dataset p ON o.order_id = p.order_id
JOIN
    olist_order_reviews_dataset r ON o.order_id = r.order_id
JOIN
    olist_order_items_dataset i ON o.order_id = i.order_id
JOIN
    olist_products_dataset x ON i.product_id = x.product_id

'''

# Execute the query and load the result directly into a new pandas DataFrame
df = pd.read_sql_query(join_query, conn)


# Close the database connection as it's no longer needed
conn.close()

# Display the first 5 rows of the final master DataFrame to verify the result
display(df.head())

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,...,freight_value,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP,00e7ee1b050b8499577073aeb2a297a1,06b8999e2fba1a1fbc88172c00ba8bc7,delivered,2017-05-16 15:05:35,2017-05-16 15:22:12,...,21.88,a9516a079e37a9c9c36b9b78b10169e8,moveis_escritorio,41.0,1141.0,1.0,8683.0,54.0,64.0,31.0
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP,29150127e6685892b6eab3eec79f59c7,18955e83d337fd6b2def6b18a428ac77,delivered,2018-01-12 20:48:24,2018-01-12 20:58:32,...,46.48,4aa6014eceb682077f9dc4bffebc05b0,utilidades_domesticas,43.0,1002.0,3.0,10150.0,89.0,15.0,40.0
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP,b2059ed67ce144a36e2aa97d2c9e9ad2,4e7b3e00288586ebd08712fdd0374a03,delivered,2018-05-19 16:07:45,2018-05-20 16:19:10,...,17.79,bd07b66896d6f1494f5b86251848ced7,moveis_escritorio,55.0,955.0,1.0,8267.0,52.0,52.0,17.0
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP,951670f92359f4fe4a63112aa7306eba,b2b6027bc5c5109e529d4dc6358b12c3,delivered,2018-03-13 16:06:38,2018-03-13 17:29:19,...,23.36,a5647c44af977b148e0a3a4751a09e2e,moveis_escritorio,48.0,1066.0,1.0,12160.0,56.0,51.0,28.0
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP,6b7d50bd145f6fc7f33cebabd7e49d0f,4f2d8ab171c80ec8364f7c12e35b23ad,delivered,2018-07-29 09:51:30,2018-07-29 10:10:09,...,22.25,9391a573abe00141c56e38d84d7d5b3b,casa_conforto,61.0,407.0,1.0,5200.0,45.0,15.0,35.0


# Data Dictionary

This section provides a detailed description of each column in the dataset, grouped by their respective entities for better understanding.

## Customer Information
* **customer_id**: An identifier for a customer related to a specific order. The same customer will have different IDs for different orders.
* **customer_unique_id**: A unique identifier for a customer across all their orders. This is the main key for a single customer.
* **customer_zip_code_prefix**: The first five digits of the customer's zip code.
* **customer_city**: The city where the customer resides.
* **customer_state**: The state where the customer resides.

## Order Information
* **order_id**: A unique identifier for each order.
* **order_status**: The current status of the order (e.g., `delivered`, `shipped`, `invoiced`).
* **order_purchase_timestamp**: The date and time when the purchase was made.
* **order_approved_at**: The date and time when the payment was approved.
* **order_delivered_carrier_date**: The date and time when the order was handed to the shipping carrier.
* **order_delivered_customer_date**: The actual delivery date of the order to the customer.
* **order_estimated_delivery_date**: The estimated delivery date that was shown to the customer at the time of purchase.

## Payment Information
* **payment_sequential**: A sequence number for orders that use multiple payment methods.
* **payment_type**: The payment method used (e.g., `credit_card`, `boleto`, `voucher`).
* **payment_installments**: The number of installments for the payment.
* **payment_value**: The total value of the payment transaction.

## Review Information
* **review_id**: A unique identifier for the order review.
* **review_score**: The score (from 1 to 5) given by the customer for the order.
* **review_comment_title**: The title of the customer's review comment (can be blank).
* **review_comment_message**: The text message of the customer's review (can be blank).
* **review_creation_date**: The date when the satisfaction survey was sent to the customer.
* **review_answer_timestamp**: The date and time when the customer submitted their review.

## Order Item Information
* **order_item_id**: A sequential identifier for items within the same order (e.g., 1, 2, 3...).
* **product_id**: A unique identifier for the product.
* **seller_id**: A unique identifier for the seller.
* **shipping_limit_date**: The seller's deadline to hand the order over to the carrier.
* **price**: The price of the individual item.
* **freight_value**: The shipping cost (freight) for the individual item.

## Product Information
* **product_category_name**: The category of the product.
* **product_name_lenght**: The number of characters in the product's name.
* **product_description_lenght**: The number of characters in the product's description.
* **product_photos_qty**: The number of published photos for the product.
* **product_weight_g**: The weight of the product in grams.
* **product_length_cm**: The length of the product in centimeters.
* **product_height_cm**: The height of the product in centimeters.
* **product_width_cm**: The width of the product in centimeters.

# Nulls and Duplicates

In [13]:
# Possible columns to be removed by 50%+ null values
null_percentage = df.isna().mean() * 100
null_percentage

customer_id                       0.000000
customer_unique_id                0.000000
customer_zip_code_prefix          0.000000
customer_city                     0.000000
customer_state                    0.000000
order_id                          0.000000
customer_id                       0.000000
order_status                      0.000000
order_purchase_timestamp          0.000000
order_approved_at                 0.012678
order_delivered_carrier_date      1.059883
order_delivered_customer_date     2.187381
order_estimated_delivery_date     0.000000
order_id                          0.000000
payment_sequential                0.000000
payment_type                      0.000000
payment_installments              0.000000
payment_value                     0.000000
review_id                         0.000000
order_id                          0.000000
review_score                      0.000000
review_comment_title             88.091958
review_comment_message           57.177873
review_crea

In [14]:
df.duplicated().sum()

np.int64(0)

# Data cleaning

In [15]:
# 1. Standardize column names
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace(r'[^\w\s]', '', regex=True)

# 2. Remove duplicate columns
df = df.loc[:, ~df.columns.duplicated()]

# 3. Remove duplicate rows
df = df.drop_duplicates()

# 4. Drop columns that are useless for the analysis
columns_to_drop = ['customer_zip_code_prefix', 'order_approved_at', 'order_delivered_carrier_date',
                   'order_delivered_customer_date', 'order_estimated_delivery_date',
                  'review_comment_title', 'review_comment_message', 'review_creation_date', 'review_answer_timestamp',
                  'order_item_id', 'seller_id', 'review_id', 'shipping_limit_date', 'product_name_lenght',
                  'product_description_lenght', 'product_photos_qty', 'product_weight_g', 'product_length_cm',
                  'product_height_cm', 'product_width_cm', 'product_id', 'order_status']  
df = df.drop(columns=columns_to_drop, errors='ignore') 

# 5. Remove columns with only 1 value (no variance)
cols_with_no_variance = [col for col in df.columns if df[col].nunique() <= 1]
df = df.drop(columns=cols_with_no_variance, errors='ignore')

# 6. Standardize text values
final_cat_cols = df.select_dtypes(include='object').columns
for col in final_cat_cols:
    df[col] = df[col].astype(str).str.strip().str.lower()
    
# 7. Date column
df['order_purchase_timestamp'] = pd.to_datetime(df['order_purchase_timestamp'])

df.head()

,customer_id,customer_unique_id,customer_city,customer_state,order_id,order_purchase_timestamp,payment_sequential,payment_type,payment_installments,payment_value,review_score,price,freight_value,product_category_name
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,franca,sp,00e7ee1b050b8499577073aeb2a297a1,2017-05-16 15:05:35,1,credit_card,2,146.87,4,124.99,21.88,moveis_escritorio
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,sao bernardo do campo,sp,29150127e6685892b6eab3eec79f59c7,2018-01-12 20:48:24,1,credit_card,8,335.48,5,289.00,46.48,utilidades_domesticas
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,sao paulo,sp,b2059ed67ce144a36e2aa97d2c9e9ad2,2018-05-19 16:07:45,1,credit_card,7,157.73,5,139.94,17.79,moveis_escritorio
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,mogi das cruzes,sp,951670f92359f4fe4a63112aa7306eba,2018-03-13 16:06:38,1,credit_card,1,173.30,5,149.94,23.36,moveis_escritorio
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,campinas,sp,6b7d50bd145f6fc7f33cebabd7e49d0f,2018-07-29 09:51:30,1,credit_card,8,252.25,5,230.00,22.25,casa_conforto


# Product Categories data

# Explanation
* Due to a high number of categories and similarity between them this step will alocate them into macro categories

In [16]:
df['product_category_name'].unique()

array(['moveis_escritorio', 'utilidades_domesticas', 'casa_conforto',
       'esporte_lazer', 'informatica_acessorios', 'none', 'brinquedos',
       'moveis_decoracao', 'automotivo', 'climatizacao', 'telefonia',
       'beleza_saude', 'ferramentas_jardim', 'pet_shop',
       'cama_mesa_banho', 'bebes', 'relogios_presentes',
       'moveis_cozinha_area_de_servico_jantar_e_jardim', 'perfumaria',
       'artes', 'papelaria', 'fashion_roupa_feminina', 'consoles_games',
       'construcao_ferramentas_iluminacao', 'alimentos_bebidas',
       'bebidas', 'cool_stuff', 'fashion_bolsas_e_acessorios',
       'casa_construcao', 'malas_acessorios', 'eletronicos',
       'eletrodomesticos_2', 'fashion_roupa_masculina', 'eletroportateis',
       'portateis_casa_forno_e_cafe', 'livros_interesse_geral',
       'eletrodomesticos', 'construcao_ferramentas_ferramentas',
       'sinalizacao_e_seguranca', 'instrumentos_musicais',
       'construcao_ferramentas_construcao', 'musica', 'fashion_calcados',
    

In [20]:
def product_categorization(category):
    """
    Function to map each product category into macro category.
    """
    categories_map = {
        'moveis_escritorio': 'Móveis e Decoração',
        'utilidades_domesticas': 'Casa e Construção',
        'casa_conforto': 'Móveis e Decoração',
        'esporte_lazer': 'Lazer e Entretenimento',
        'informatica_acessorios': 'Eletrodomésticos e Eletrônicos',
        'none': 'Outros',
        'brinquedos': 'Infantil',
        'moveis_decoracao': 'Móveis e Decoração',
        'automotivo': 'Automotivo',
        'climatizacao': 'Eletrodomésticos e Eletrônicos',
        'telefonia': 'Eletrodomésticos e Eletrônicos',
        'beleza_saude': 'Saúde e Beleza',
        'ferramentas_jardim': 'Casa e Construção',
        'pet_shop': 'Pet Shop',
        'cama_mesa_banho': 'Móveis e Decoração',
        'bebes': 'Infantil',
        'relogios_presentes': 'Moda e Acessórios',
        'moveis_cozinha_area_de_servico_jantar_e_jardim': 'Móveis e Decoração',
        'perfumaria': 'Saúde e Beleza',
        'artes': 'Lazer e Entretenimento',
        'papelaria': 'Papelaria e Escritório',
        'fashion_roupa_feminina': 'Moda e Acessórios',
        'consoles_games': 'Eletrodomésticos e Eletrônicos',
        'construcao_ferramentas_iluminacao': 'Casa e Construção',
        'alimentos_bebidas': 'Alimentos e Bebidas',
        'bebidas': 'Alimentos e Bebidas',
        'cool_stuff': 'Outros',
        'fashion_bolsas_e_acessorios': 'Moda e Acessórios',
        'casa_construcao': 'Casa e Construção',
        'malas_acessorios': 'Moda e Acessórios',
        'eletronicos': 'Eletrodomésticos e Eletrônicos',
        'eletrodomesticos_2': 'Eletrodomésticos e Eletrônicos',
        'fashion_roupa_masculina': 'Moda e Acessórios',
        'eletroportateis': 'Eletrodomésticos e Eletrônicos',
        'portateis_casa_forno_e_cafe': 'Eletrodomésticos e Eletrônicos',
        'livros_interesse_geral': 'Lazer e Entretenimento',
        'eletrodomesticos': 'Eletrodomésticos e Eletrônicos',
        'construcao_ferramentas_ferramentas': 'Casa e Construção',
        'sinalizacao_e_seguranca': 'Casa e Construção',
        'instrumentos_musicais': 'Lazer e Entretenimento',
        'construcao_ferramentas_construcao': 'Casa e Construção',
        'musica': 'Lazer e Entretenimento',
        'fashion_calcados': 'Moda e Acessórios',
        'industria_comercio_e_negocios': 'Indústria e Comércio',
        'fashion_underwear_e_moda_praia': 'Moda e Acessórios',
        'dvds_blu_ray': 'Lazer e Entretenimento',
        'construcao_ferramentas_seguranca': 'Casa e Construção',
        'alimentos': 'Alimentos e Bebidas',
        'telefonia_fixa': 'Eletrodomésticos e Eletrônicos',
        'moveis_sala': 'Móveis e Decoração',
        'tablets_impressao_imagem': 'Eletrodomésticos e Eletrônicos',
        'market_place': 'Outros',
        'artigos_de_natal': 'Móveis e Decoração',
        'agro_industria_e_comercio': 'Indústria e Comércio',
        'construcao_ferramentas_jardim': 'Casa e Construção',
        'pcs': 'Eletrodomésticos e Eletrônicos',
        'moveis_quarto': 'Móveis e Decoração',
        'audio': 'Eletrodomésticos e Eletrônicos',
        'livros_importados': 'Lazer e Entretenimento',
        'livros_tecnicos': 'Lazer e Entretenimento',
        'artigos_de_festas': 'Lazer e Entretenimento',
        'portateis_cozinha_e_preparadores_de_alimentos': 'Eletrodomésticos e Eletrônicos',
        'pc_gamer': 'Eletrodomésticos e Eletrônicos',
        'moveis_colchao_e_estofado': 'Móveis e Decoração',
        'la_cuisine': 'Alimentos e Bebidas',
        'flores': 'Móveis e Decoração',
        'fraldas_higiene': 'Infantil',
        'cine_foto': 'Eletrodomésticos e Eletrônicos',
        'cds_dvds_musicais': 'Lazer e Entretenimento',
        'fashion_esporte': 'Moda e Acessórios',
        'casa_conforto_2': 'Móveis e Decoração',
        'artes_e_artesanato': 'Lazer e Entretenimento',
        'fashion_roupa_infanto_juvenil': 'Infantil',
        'seguros_e_servicos': 'Outros'
    }
    return categories_map.get(category, 'Outros') 
df['macro_categoria'] = df['product_category_name'].apply(product_categorization)
df.head()

,customer_id,customer_unique_id,customer_city,customer_state,order_id,order_purchase_timestamp,payment_sequential,payment_type,payment_installments,payment_value,review_score,price,freight_value,product_category_name,macro_categoria
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,franca,sp,00e7ee1b050b8499577073aeb2a297a1,2017-05-16 15:05:35,1,credit_card,2,146.87,4,124.99,21.88,moveis_escritorio,Móveis e Decoração
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,sao bernardo do campo,sp,29150127e6685892b6eab3eec79f59c7,2018-01-12 20:48:24,1,credit_card,8,335.48,5,289.00,46.48,utilidades_domesticas,Casa e Construção
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,sao paulo,sp,b2059ed67ce144a36e2aa97d2c9e9ad2,2018-05-19 16:07:45,1,credit_card,7,157.73,5,139.94,17.79,moveis_escritorio,Móveis e Decoração
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,mogi das cruzes,sp,951670f92359f4fe4a63112aa7306eba,2018-03-13 16:06:38,1,credit_card,1,173.30,5,149.94,23.36,moveis_escritorio,Móveis e Decoração
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,campinas,sp,6b7d50bd145f6fc7f33cebabd7e49d0f,2018-07-29 09:51:30,1,credit_card,8,252.25,5,230.00,22.25,casa_conforto,Móveis e Decoração


# Save df

In [18]:

# Path to the folder where the file will be saved
folder_path = r"C:\Users\win\Desktop\PROJETO\dados"

# File name
file_name = "df_clean.csv"

# Join to create the full path
full_path = os.path.join(folder_path, file_name)

# Save the DataFrame to CSV format
df.to_csv(full_path, index=False, encoding='utf-8-sig')

print(f"✅ Dataset saved successfully at: {full_path}")


✅ Dataset saved successfully at: C:\Users\win\Desktop\PROJETO\dados\df_clean.csv
